Import Libraries

In [10]:
import os
import PyPDF2
import faiss
import torch
import numpy as np
from sentence_transformers import SentenceTransformer
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

RAG Application Class

In [11]:
class RAGApplication:
    def __init__(self, pdf_path):
        self.pdf_path = pdf_path
        self.embedder = SentenceTransformer('all-MiniLM-L6-v2')
        self.tokenizer = AutoTokenizer.from_pretrained("google/flan-t5-base")
        self.model = AutoModelForSeq2SeqLM.from_pretrained("google/flan-t5-base")
        self.chunks = self.process_pdf()
        self.index = self.create_index()

    def process_pdf(self):
        chunks = []
        with open(self.pdf_path, 'rb') as file:
            reader = PyPDF2.PdfReader(file)
            for page in reader.pages:
                text = page.extract_text()
                chunks.extend([text[i:i+512] for i in range(0, len(text), 512)])
        return chunks

    def create_index(self):
        embeddings = self.embedder.encode(self.chunks)
        dimension = embeddings.shape[1]
        index = faiss.IndexFlatL2(dimension)
        index.add(np.array(embeddings).astype('float32'))
        return index

    def retrieve_relevant_chunks(self, query, k=3):
        query_embedding = self.embedder.encode([query])
        _, indices = self.index.search(np.array(query_embedding).astype('float32'), k)
        return [self.chunks[i] for i in indices[0]]

    def answer_question(self, question):
        relevant_chunks = self.retrieve_relevant_chunks(question)
        context = " ".join(relevant_chunks)
        
        input_text = f"Answer the following question based on the given context. If the answer is not in the context, say 'I don't have enough information to answer this question.'/n/nContext: {context}/n/nQuestion: {question}/n/nAnswer:"
        
        input_ids = self.tokenizer(input_text, return_tensors="pt").input_ids
        outputs = self.model.generate(input_ids, max_length=150, num_return_sequences=1, temperature=0.7)
        answer = self.tokenizer.decode(outputs[0], skip_special_tokens=True)
        
        summary = self.summarize_result(question, answer, context)
        return answer, summary

    def summarize_result(self, question, answer, context):
        summary_prompt = f"Summarize the following question and answer pair, mentioning the key points from the context:/n/nQuestion: {question}/n/nAnswer: {answer}/n/nContext: {context}/n/nSummary:"
        
        input_ids = self.tokenizer(summary_prompt, return_tensors="pt").input_ids
        outputs = self.model.generate(input_ids, max_length=100, num_return_sequences=1, temperature=0.7)
        summary = self.tokenizer.decode(outputs[0], skip_special_tokens=True)
        return summary

Run the RAG Application

In [16]:
if __name__ == "__main__":
    pdf_path = "C:/Users/gurme/Desktop/PDFs/LawsOfChess.pdf"
    rag_app = RAGApplication(pdf_path)
    
    while True:
        question = input("Enter your question (or 'quit' to exit): ")
        if question.lower() == 'quit':
            break
        
        answer, summary = rag_app.answer_question(question)
        print(f"\n{question}?")
        print(f"\nAnswer: {answer}\n")
        print(f"\n*************************************\n")
        # print(f"Summary: {summary}/n")

c:\Users\gurme\anaconda3\Lib\site-packages\transformers\generation\configuration_utils.py:567: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.7` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(



How long is a chess game. Just give me the time.

Answer: The default time is 0 minutes


*************************************


Does chess have a queen piece. Yes or not?

Answer: Yes


*************************************


Does chess have joker as a piece. Yes or no?

Answer: No


*************************************

